In [1]:
# Import Zipline, the open source backester, and a few other libraries that we will use
import zipline
from zipline import TradingAlgorithm
from zipline.data.loader import load_bars_from_yahoo
from zipline.api import order_target, symbol, history, add_history, schedule_function, date_rules, time_rules

from datetime import datetime
import pytz

import numpy as np

import pyfolio as pf

In [2]:
def initialize(context):
    # Register 2 histories that track daily prices,
    # one with a 100 window and one with a 300 day window
    add_history(100, '1d', 'price')
    add_history(300, '1d', 'price')

    context.i = 0


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = history(100, '1d', 'price').mean()
    long_mavg = history(300, '1d', 'price').mean()

    sym = symbol('AAPL')

    # Trading logic
    if short_mavg[sym] > long_mavg[sym]:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(sym, 100)
    elif short_mavg[sym] < long_mavg[sym]:
        order_target(sym, 0)

In [3]:
algo_obj = TradingAlgorithm(initialize=initialize, handle_data=handle_data)

[2015-07-10 18:30:21.077684] WARNING: Loader: No benchmark data found for date range.
start_date=2015-07-10 00:00:00+00:00, end_date=2015-07-10 18:30:21.011172, url=http://ichart.finance.yahoo.com/table.csv?a=6&c=2015&b=10&e=10&d=6&g=d&f=2015&s=%5EGSPC


In [4]:
start = datetime(2015, 5, 1, 0, 0, 0, 0, pytz.utc)
end = datetime(2015, 7, 1, 0, 0, 0, 0, pytz.utc)
data = load_bars_from_yahoo(stocks=['AAPL'], start=start, end=end)

AAPL


In [5]:
backtest = algo_obj.run(data)

[2015-07-10 18:30:30.698501] INFO: Performance: Simulated 43 trading days out of 43.
[2015-07-10 18:30:30.699157] INFO: Performance: first open: 2015-05-01 13:31:00+00:00
[2015-07-10 18:30:30.699624] INFO: Performance: last close: 2015-07-01 20:00:00+00:00


In [27]:
df_rets = backtest[0].returns
df_rets.groupby(df_rets.index.date).last()

2015-05-01    0
2015-05-04    0
2015-05-05    0
2015-05-06    0
2015-05-07    0
2015-05-08    0
2015-05-11    0
2015-05-12    0
2015-05-13    0
2015-05-14    0
2015-05-15    0
2015-05-18    0
2015-05-19    0
2015-05-20    0
2015-05-21    0
2015-05-22    0
2015-05-26    0
2015-05-27    0
2015-05-28    0
2015-05-29    0
2015-06-01    0
2015-06-02    0
2015-06-03    0
2015-06-04    0
2015-06-05    0
2015-06-08    0
2015-06-09    0
2015-06-10    0
2015-06-11    0
2015-06-12    0
2015-06-15    0
2015-06-16    0
2015-06-17    0
2015-06-18    0
2015-06-19    0
2015-06-22    0
2015-06-23    0
2015-06-24    0
2015-06-25    0
2015-06-26    0
2015-06-29    0
2015-06-30    0
2015-07-01    0
Name: returns, dtype: float64

In [14]:
type(backtest[0].returns.index[0])

pandas.tslib.Timestamp

In [26]:
pf.create_full_tear_sheet(df_rets)

Entire data start date: 2015-05-01 20:00:00
Entire data end date: 2015-07-01 20:00:00


Out-of-Sample Months: 1
Backtest Months: 1


KeyError: "None of [DatetimeIndex(['2015-05-01 20:00:00', '2015-05-04 20:00:00',\n               '2015-05-05 20:00:00', '2015-05-06 20:00:00',\n               '2015-05-07 20:00:00', '2015-05-08 20:00:00',\n               '2015-05-11 20:00:00', '2015-05-12 20:00:00',\n               '2015-05-13 20:00:00', '2015-05-14 20:00:00',\n               '2015-05-15 20:00:00', '2015-05-18 20:00:00',\n               '2015-05-19 20:00:00', '2015-05-20 20:00:00',\n               '2015-05-21 20:00:00', '2015-05-22 20:00:00',\n               '2015-05-26 20:00:00', '2015-05-27 20:00:00',\n               '2015-05-28 20:00:00', '2015-05-29 20:00:00',\n               '2015-06-01 20:00:00'],\n              dtype='datetime64[ns]', freq=None, tz=None)] are in the [index]"

In [ ]:
"""
# Mean reverting algo. 

# Put any initialization logic here.  The context object will be passed to
# the other methods in your algorithm.
def initialize(context):
    context.stocks = symbols('COKE', 'PEP')
    schedule_function(weekly_rebalance, date_rules.week_start(), time_rules.market_open(hours = 1))
    add_history(60, '1d', 'price')
    
def weekly_rebalance(context,data):        
    # calculate the price ratio
    context.price_ratio = context.coke_price/context.pepsi_price
    
    # calculate the standard deviation for the stock's prices
    history_frame = history(60, '1d', 'price')
    trailing_price_ratio = history_frame[symbol('COKE')]/history_frame[symbol('PEP')]
    
    std_ratio = trailing_price_ratio.std()
    
    # order_target's only consider the status of filled orders
    if get_open_orders():
        return
    
    #enter a trade if the price ratio is more/less than 2 stdev of expected ratio
    if context.price_ratio > 2*std_ratio:
        #go long on the under-valued stock
        order_target_percent(symbol('COKE'), 0.5)     
        # and go short on the over-valued stock
        order_target_percent(symbol('PEP'), -0.5)
        
    elif context.price_ratio < 2*std_ratio:
        #go long on the under-valued stock
        order_target_percent(symbol('PEP'), 0.5)
        # and go short on the over-valued stock
        order_target_percent(symbol('COKE'), -0.5)
            
# Will be called on every trade event for the securities you specify. 
def handle_data(context, data):  
    context.coke_price = data[symbol('COKE')].price
    context.pepsi_price = data[symbol('PEP')].price"""